# Distributing ScikitLearn applications on the Slashbin cluster using Ray

## Installation

### Install Ray

In [1]:
pip install "ray[default]"

Note: you may need to restart the kernel to use updated packages.


### Installing NERSC's slurm-magic to interact with SLURM on Jupyter

In [2]:
pip install git+https://github.com/NERSC/slurm-magic.git pandas

  Cloning https://github.com/NERSC/slurm-magic.git to /tmp/pip-req-build-aklmtzix
  Running command git clone --filter=blob:none -q https://github.com/NERSC/slurm-magic.git /tmp/pip-req-build-aklmtzix
  Resolved https://github.com/NERSC/slurm-magic.git to commit 4c708cc137cb9f4bd5b44cf26837b466d9bf7b65
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
load_ext slurm_magic

### Installing Example requirements

In [4]:
pip install scikit-learn numpy joblib

Note: you may need to restart the kernel to use updated packages.


## Creating our first Scikit-learn script with Ray backend
(taken from [Ray documentation](https://docs.ray.io/en/latest/joblib.html))

In [5]:
%%writefile sklearn_ray.py
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

digits = load_digits()
param_space = {
    'C': np.logspace(-6, 6, 30),
    'gamma': np.logspace(-8, 8, 30),
    'tol': np.logspace(-4, -1, 30),
    'class_weight': [None, 'balanced'],
}
model = SVC(kernel='rbf')
search = RandomizedSearchCV(model, param_space, cv=5, n_iter=300, verbose=10)

import os
import ray
import joblib
from ray.util.joblib import register_ray

ray.init(address=os.environ["ip_head"])
register_ray()

with joblib.parallel_backend('ray'):
    search.fit(digits.data, digits.target)

Overwriting sklearn_ray.py


## Launching Ray with SBATCH

Since starting a SLURM interactive job in Jupyter is more or less equivalent to running an SBATCH script in Jupyter, we will be using SBATCH. If you start Jupyter within the SLURM allocation, then the steps presented here can be executed interactively.

The below SBATCH script will need to be modified according to your needs. The following script runs the sklearn_ray.py (created in the previous step) and will execute on the cluster as is.

In [6]:
%%sbatch --output ray_example.out
#!/bin/bash                                                                                                                                  
#SBATCH --mem=230G
#SBATCH --cpus-per-task=64                                                       
#SBATCH --tasks-per-node=1
#SBATCH --nodes=8

# source venv. **THIS WILL NEED TO BE CHANGED TO YOUR VENVS NAME**
source venv/bin/activate

## The following is taken directly from the Ray docs (unmodified) https://docs.ray.io/en/latest/cluster/slurm.html
## Get the HEAD IP
# Getting the node names
nodes=$(scontrol show hostnames "$SLURM_JOB_NODELIST")
nodes_array=($nodes)

head_node=${nodes_array[0]}
head_node_ip=$(srun --nodes=1 --ntasks=1 -w "$head_node" hostname --ip-address)

# if we detect a space character in the head node IP, we'll
# convert it to an ipv4 address. This step is optional.
if [[ "$head_node_ip" == *" "* ]]; then
IFS=' ' read -ra ADDR <<<"$head_node_ip"
if [[ ${#ADDR[0]} -gt 16 ]]; then
  head_node_ip=${ADDR[1]}
else
  head_node_ip=${ADDR[0]}
fi
echo "IPV6 address detected. We split the IPV4 address as $head_node_ip"
fi

## Starting the head node
port=6379
ip_head=$head_node_ip:$port
export ip_head
echo "IP Head: $ip_head"

echo "Starting HEAD at $head_node"
srun --nodes=1 --ntasks=1 -w "$head_node" \
    ray start --head --node-ip-address="$head_node_ip" --port=$port \
    --num-cpus "${SLURM_CPUS_PER_TASK}" --include-dashboard True --dashboard-host "$head_node_ip" --block &
    
## Starting the workers
# optional, though may be useful in certain versions of Ray < 1.0.
sleep 10

# number of nodes other than the head node
worker_num=$((SLURM_JOB_NUM_NODES - 1))

for ((i = 1; i <= worker_num; i++)); do
    node_i=${nodes_array[$i]}
    echo "Starting WORKER $i at $node_i"
    srun --nodes=1 --ntasks=1 -w "$node_i" \
        ray start --address "$ip_head" \
        --num-cpus "${SLURM_CPUS_PER_TASK}" --block &
    sleep 5
done

### **Running sklearn_ray.py **
    
python -u sklearn_ray.py
    

'Submitted batch job 18821\n'

## Ray Dashboard

The Dashboard is generated on the headnode. In our case, this would be on comp01. The default port used by Ray for the dashboard is 8265.
To verify that this is accurate, we can check the log output of our SLURM job as the dashboard IP will be written in the first few lines

Here we set the output filename to be `ray_example.out`. Normally slurm logfiles are by default formatted to be in the form `slurm_<jobid>.out`

Opening the URL in a new browser window will load the dashboard.

In [7]:
%%bash 
sleep 15
cat ray_example.out

IP Head: 10.1.1.13:6379
Starting HEAD at comp01
2022-01-19 13:11:47,726	INFO services.py:1338 -- View the Ray dashboard at http://10.1.1.13:8265
2022-01-19 13:11:45,082	INFO scripts.py:612 -- Local node IP: 10.1.1.13
2022-01-19 13:11:47,865	SUCC scripts.py:651 -- --------------------
2022-01-19 13:11:47,865	SUCC scripts.py:652 -- Ray runtime started.
2022-01-19 13:11:47,866	SUCC scripts.py:653 -- --------------------
2022-01-19 13:11:47,866	INFO scripts.py:655 -- Next steps
2022-01-19 13:11:47,866	INFO scripts.py:656 -- To connect to this Ray runtime from another node, run
2022-01-19 13:11:47,866	INFO scripts.py:660 --   ray start --address='10.1.1.13:6379' --redis-password='5241590000000000'
2022-01-19 13:11:47,866	INFO scripts.py:665 -- Alternatively, use the following Python code:
2022-01-19 13:11:47,867	INFO scripts.py:668 -- import ray
2022-01-19 13:11:47,867	INFO scripts.py:669 -- ray.init(address='auto', _redis_password='5241590000000000')
2022-01-19 13:11:47,867	INFO scripts.py